In [63]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain import hub
load_dotenv()

loader = PyPDFLoader("unsu.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, 
    chunk_overlap=20, 
    length_function=len
)
texts = text_splitter.split_documents(pages)

langchain_core.documents.base.Document

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
db = Chroma.from_documents(texts, embeddings) 
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
question = "이 책의 저자는 누구인가?" 
question = "아내가 먹고 싶어하던 음식은 무엇인가?" 
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), 
    llm = llm) 
docs = retriever_from_llm.invoke(question)
prompt = hub.pull("rlm/rag-prompt")
def format_docs(docs): 
    return "\n\n".join(doc.page_content for doc in docs) 
rag_chain = (
    {"context": retriever_from_llm | format_docs, 
     "question": RunnablePassthrough()}
     | prompt
     | llm
     | StrOutputParser() 
)
rag_chain.invoke("이 소설의 제목은 운수 좋은 날 이다 작가는 누구인가?")
prompt